<a href="https://colab.research.google.com/github/MHusni1604/Pengantar-Anlisis-Data-Panel/blob/main/Tugas_Minggu_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis Dampak Trump Tariffs Terhadap Harga Saham LQ45
---
![Trump Tariffs](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Trump_showing_a_chart_with_reciprocal_tariffs_%28cropped%29.jpg/500px-Trump_showing_a_chart_with_reciprocal_tariffs_%28cropped%29.jpg)

# Pendahuluan

Pada tanggal 2 April 2025, pemerintahan Presiden Donald Trump mengumumkan kebijakan baru terkait tarif impor yang berdampak signifikan pada dinamika ekonomi global. Kebijakan yang dikenal sebagai **"Trump Tariffs"** ini menaikkan tarif rata-rata barang impor ke Amerika Serikat dari sekitar 2,5% menjadi 27% [\[1\]](https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration). Kenaikan tarif tersebut memicu gejolak di pasar global, termasuk memberikan tekanan pada pasar keuangan negara berkembang seperti Indonesia.

Indeks **LQ45** merupakan indeks harga saham yang mencakup 45 saham paling likuid di Bursa Efek Indonesia (BEI) dan menjadi indikator utama dalam menilai respon pasar domestik terhadap perubahan kondisi global [\[2\]](https://id.wikipedia.org/wiki/Indeks_LQ45). Harga saham dalam indeks ini dipengaruhi oleh sentimen internasional serta kondisi makroekonomi secara umum. Oleh karena itu, penting untuk menganalisis dampak kebijakan tarif Trump terhadap pergerakan harga saham di indeks LQ45.

Penelitian ini bertujuan mengukur pengaruh kebijakan **"Trump Tariffs"** terhadap harga saham perusahaan-perusahaan yang tergabung dalam indeks LQ45. Variabel dependen yang digunakan adalah harga penutupan saham harian, sedangkan variabel independen mencakup suku bunga acuan Bank Indonesia (BI rate), tingkat inflasi, nilai tukar Rupiah terhadap Dolar Amerika Serikat, serta variabel dummy yang menandai periode penerapan tarif tersebut. Dengan menggunakan pendekatan data panel yang mengkombinasikan dimensi lintas perusahaan dan lintas waktu, analisis ini diharapkan dapat menangkap dinamika kompleks dalam respons harga saham terhadap tekanan eksternal dan kondisi makroekonomi domestik [\[3\]](https://doi.org/10.24912/jmk.v4i1.17118).

# Metodologi

## Sumber Data  
Penelitian ini menggunakan pendekatan kuantitatif dengan metode regresi data panel dinamis untuk menganalisis pengaruh kebijakan tarif impor Amerika Serikat tahun 2025 (Tarif Trump 2025) terhadap harga saham perusahaan yang tergabung dalam Indeks LQ45 di Bursa Efek Indonesia. Data yang digunakan merupakan data sekunder yang diperoleh dari dua sumber utama, yaitu [Yahoo Finance](https://finance.yahoo.com) untuk data harga penutupan saham harian dan nilai tukar Rupiah terhadap Dolar AS, serta [situs resmi Bank Indonesia](https://www.bi.go.id) untuk data suku bunga acuan (BI Rate) dan tingkat inflasi.

## Spesifikasi Model  
Model panel dinamis dipilih untuk menangkap efek ketergantungan harga saham pada periode sebelumnya (lagged dependent variable) serta efek variabel independen makroekonomi dan kebijakan tarif terhadap harga saham. Pendekatan ini memungkinkan analisis yang lebih realistis terhadap dinamika harga saham yang umumnya memiliki sifat autoregresif.

Model regresi panel dinamis yang digunakan dapat dirumuskan sebagai berikut:

$$
Y_{it} = \alpha_{i} + \rho Y_{i,t-1} + \beta_1 (\text{BI-Rate})_{t} + \beta_2 (\text{Rate Inflasi})_{t} + \beta_3 (\text{Kurs})_{t} + \beta_4 (\text{Dummy Tarif})_{t} + \epsilon_{it}
$$

Keterangan:  
- $Y_{it}$: Harga penutupan saham perusahaan $i$ pada waktu $t$  
- $Y_{i,t-1}$: Harga penutupan saham perusahaan $i$ pada periode sebelumnya ($t-1$)  
- $\alpha_i$: Efek tetap (fixed effect) khusus untuk masing-masing perusahaan  
- $\rho, \beta_1, ..., \beta_4$: Koefisien regresi  
- $\epsilon_{it}$: Term error  

Penggunaan model panel dinamis ini memungkinkan untuk menangani adanya dinamika harga saham yang dipengaruhi oleh nilai historisnya sendiri, sekaligus memperhitungkan heterogenitas antar perusahaan dalam indeks LQ45. Estimasi model ini dilakukan menggunakan metode Generalized Method of Moments (GMM) untuk mengatasi potensi masalah endogenitas yang muncul dari variabel lagged dependent dan variabel independen lainnya.

Dengan pendekatan ini, diharapkan hasil analisis dapat memberikan gambaran yang lebih akurat mengenai dampak kebijakan tarif Trump 2025 dan kondisi makroekonomi terhadap pergerakan harga saham di pasar modal Indonesia.


# Hasil dan Pembahasan

## Import Library

In [1]:
pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.6 MB/s eta 0:00:00


In [2]:
# Library umum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pengambilan data finansial
import yfinance as yf

# Statistik dan ekonometrika
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

# Model panel data
from linearmodels.panel import PanelOLS

# Machine learning preprocessing dan reduksi dimensi
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Data Loading

Mengambil data saham LQ45 dari laman Yahoo Finance

In [3]:
tickers = [
    "ACES.JK", "ADMR.JK", "ADRO.JK", "AKRA.JK", "AMMN.JK", "AMRT.JK", "ANTM.JK", "ARTO.JK",
    "ASII.JK", "BBCA.JK", "BBNI.JK", "BBRI.JK", "BBTN.JK", "BMRI.JK", "BRIS.JK", "BRPT.JK",
    "CPIN.JK", "CTRA.JK", "ESSA.JK", "EXCL.JK", "GOTO.JK", "ICBP.JK", "INCO.JK", "INDF.JK",
    "INKP.JK", "ISAT.JK", "ITMG.JK", "JPFA.JK", "JSMR.JK", "KLBF.JK", "MAPA.JK", "MAPI.JK",
    "MBMA.JK", "MDKA.JK", "MEDC.JK", "PGAS.JK", "PGEO.JK", "PTBA.JK", "SIDO.JK", "SMGR.JK",
    "SMRA.JK", "TLKM.JK", "TOWR.JK", "UNTR.JK", "UNVR.JK"
]

data = yf.download(tickers, start="2025-01-01", end="2025-05-31")['Close']

data_panel = data.stack().reset_index()
data_panel.columns = ['Date', 'Ticker', 'Close']

df_saham = data_panel.copy()
df_saham['Date'] = pd.to_datetime(df_saham['Date'])

df_saham

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  45 of 45 completed


,Date,Ticker,Close
0,2025-01-02,ACES.JK,755.000000
1,2025-01-02,ADMR.JK,1165.000000
2,2025-01-02,ADRO.JK,2530.000000
3,2025-01-02,AKRA.JK,1110.232544
4,2025-01-02,AMMN.JK,8475.000000
...,...,...,...
4090,2025-05-28,SMRA.JK,404.000000
4091,2025-05-28,TLKM.JK,2820.000000
4092,2025-05-28,TOWR.JK,585.000000
4093,2025-05-28,UNTR.JK,22450.000000


Mengambil data BI-rate dari laman resmi BI

In [4]:
df_birate = {
    'Date' : ['2024-12-18', '2025-01-15', '2025-02-19', '2025-03-19', '2025-04-23', '2025-05-21'],
    'BI-Rate' : [0.0600, 0.0575, 0.0575, 0.0575, 0.0575, 0.0550]
}

df_birate = pd.DataFrame(df_birate)
df_birate['Date'] = pd.to_datetime(df_birate['Date'])
df_birate['Start Date'] = df_birate['Date']
df_birate['End Date'] = df_birate['Date'].shift(-1)

last_end_date = df_saham['Date'].max()
df_birate.loc[df_birate.index[-1], 'End Date'] = last_end_date

df_birate

,Date,BI-Rate,Start Date,End Date
0,2024-12-18,0.0600,2024-12-18,2025-01-15
1,2025-01-15,0.0575,2025-01-15,2025-02-19
2,2025-02-19,0.0575,2025-02-19,2025-03-19
3,2025-03-19,0.0575,2025-03-19,2025-04-23
4,2025-04-23,0.0575,2025-04-23,2025-05-21
5,2025-05-21,0.0550,2025-05-21,2025-05-28


In [5]:
wide_df = df_saham.reset_index().pivot(index='Date', columns='Ticker', values='Close').reset_index()

def assign_value(date):
    for i in range(len(df_birate)):
        if df_birate.loc[i, 'Start Date'] <= date <= df_birate.loc[i, 'End Date']:
            return df_birate.loc[i, 'BI-Rate']
    return None

wide_df['BI-Rate'] = wide_df['Date'].apply(assign_value)
df_saham = wide_df.melt(id_vars=['Date', 'BI-Rate'], var_name='Ticker', value_name='Close')

df_saham

,Date,BI-Rate,Ticker,Close
0,2025-01-02,0.060,ACES.JK,755.0
1,2025-01-03,0.060,ACES.JK,755.0
2,2025-01-06,0.060,ACES.JK,755.0
3,2025-01-07,0.060,ACES.JK,735.0
4,2025-01-08,0.060,ACES.JK,705.0
...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0
4091,2025-05-23,0.055,UNVR.JK,1785.0
4092,2025-05-26,0.055,UNVR.JK,1765.0
4093,2025-05-27,0.055,UNVR.JK,1740.0


Mengambil data tingkat inflasi dari laman resmi BI

In [6]:
df_inflasi = {
    'Date' : ['2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01'],
    'Rate Inflasi' : [0.0160, 0.0195, 0.0103, -0.0009, 0.0076]
}

df_inflasi = pd.DataFrame(df_inflasi)
df_inflasi['Date'] = pd.to_datetime(df_inflasi['Date'])

df_inflasi

,Date,Rate Inflasi
0,2025-01-01,0.0160
1,2025-02-01,0.0195
2,2025-03-01,0.0103
3,2025-04-01,-0.0009
4,2025-05-01,0.0076


In [7]:
df_saham['Month'] = df_saham['Date'].dt.month
df_inflasi['Month'] = df_inflasi['Date'].dt.month
df_saham = df_saham.merge(df_inflasi[['Month', 'Rate Inflasi']], on='Month', how='left')
df_saham.drop(columns=['Month'], inplace=True)

Mengambil data rate nilai tukar dollar dari laman Yahoo Finance

In [8]:
kurs = yf.download(
    "USDIDR=X",
    start=wide_df['Date'].min().strftime('%Y-%m-%d'),
    end=(wide_df['Date'].max() + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
)

df_kurs = kurs[['Close']].reset_index()
df_kurs.columns = ['Date', 'USD_IDR']

df_kurs

[*********************100%***********************]  1 of 1 completed


,Date,USD_IDR
0,2025-01-02,16230.500000
1,2025-01-03,16201.099609
2,2025-01-06,16178.799805
3,2025-01-07,16179.099609
4,2025-01-08,16157.700195
...,...,...
98,2025-05-22,16347.599609
99,2025-05-23,16356.900391
100,2025-05-26,16211.000000
101,2025-05-27,16243.500000


In [9]:
df_saham = df_saham.merge(df_kurs, left_on='Date', right_on='Date', how='left')
df_saham

,Date,BI-Rate,Ticker,Close,Rate Inflasi,USD_IDR
0,2025-01-02,0.060,ACES.JK,755.0,0.0160,16230.500000
1,2025-01-03,0.060,ACES.JK,755.0,0.0160,16201.099609
2,2025-01-06,0.060,ACES.JK,755.0,0.0160,16178.799805
3,2025-01-07,0.060,ACES.JK,735.0,0.0160,16179.099609
4,2025-01-08,0.060,ACES.JK,705.0,0.0160,16157.700195
...,...,...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0,0.0076,16347.599609
4091,2025-05-23,0.055,UNVR.JK,1785.0,0.0076,16356.900391
4092,2025-05-26,0.055,UNVR.JK,1765.0,0.0076,16211.000000
4093,2025-05-27,0.055,UNVR.JK,1740.0,0.0076,16243.500000


In [10]:
df_saham['Trump Tariffs'] = (df_saham.Date >= pd.to_datetime('2025-04-05')).astype(int)
df_saham

,Date,BI-Rate,Ticker,Close,Rate Inflasi,USD_IDR,Trump Tariffs
0,2025-01-02,0.060,ACES.JK,755.0,0.0160,16230.500000,0
1,2025-01-03,0.060,ACES.JK,755.0,0.0160,16201.099609,0
2,2025-01-06,0.060,ACES.JK,755.0,0.0160,16178.799805,0
3,2025-01-07,0.060,ACES.JK,735.0,0.0160,16179.099609,0
4,2025-01-08,0.060,ACES.JK,705.0,0.0160,16157.700195,0
...,...,...,...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0,0.0076,16347.599609,1
4091,2025-05-23,0.055,UNVR.JK,1785.0,0.0076,16356.900391,1
4092,2025-05-26,0.055,UNVR.JK,1765.0,0.0076,16211.000000,1
4093,2025-05-27,0.055,UNVR.JK,1740.0,0.0076,16243.500000,1


In [11]:
df_saham.sort_values(by=['Ticker', 'Date'], inplace=True)
df_saham.set_index(['Ticker', 'Date'], inplace=True)
df_saham

BI-Rate   Close  Rate Inflasi       USD_IDR  Trump Tariffs
Ticker  Date                                                                  
ACES.JK 2025-01-02    0.060   755.0        0.0160  16230.500000              0
        2025-01-03    0.060   755.0        0.0160  16201.099609              0
        2025-01-06    0.060   755.0        0.0160  16178.799805              0
        2025-01-07    0.060   735.0        0.0160  16179.099609              0
        2025-01-08    0.060   705.0        0.0160  16157.700195              0
...                     ...     ...           ...           ...            ...
UNVR.JK 2025-05-22    0.055  1815.0        0.0076  16347.599609              1
        2025-05-23    0.055  1785.0        0.0076  16356.900391              1
        2025-05-26    0.055  1765.0        0.0076  16211.000000              1
        2025-05-27    0.055  1740.0        0.0076  16243.500000              1
        2025-05-28    0.055  1730.0        0.0076  16279.000000              1

[4095 rows x 5 columns]

## Model Development

Data Lengkap setelah disatukan

In [12]:
df_saham.dropna(inplace=True)
df_saham

BI-Rate   Close  Rate Inflasi       USD_IDR  Trump Tariffs
Ticker  Date                                                                  
ACES.JK 2025-01-02    0.060   755.0        0.0160  16230.500000              0
        2025-01-03    0.060   755.0        0.0160  16201.099609              0
        2025-01-06    0.060   755.0        0.0160  16178.799805              0
        2025-01-07    0.060   735.0        0.0160  16179.099609              0
        2025-01-08    0.060   705.0        0.0160  16157.700195              0
...                     ...     ...           ...           ...            ...
UNVR.JK 2025-05-22    0.055  1815.0        0.0076  16347.599609              1
        2025-05-23    0.055  1785.0        0.0076  16356.900391              1
        2025-05-26    0.055  1765.0        0.0076  16211.000000              1
        2025-05-27    0.055  1740.0        0.0076  16243.500000              1
        2025-05-28    0.055  1730.0        0.0076  16279.000000              1

[4050 rows x 5 columns]

Ringakasan dari data yang digunakan

In [13]:
df_saham.describe()

,BI-Rate,Close,Rate Inflasi,USD_IDR,Trump Tariffs
count,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000
mean,0.057639,3494.110791,0.011171,16430.656630,0.355556
std,0.001011,4818.931660,0.006871,208.341001,0.478740
min,0.055000,64.000000,-0.000900,16157.700195,0.000000
25%,0.057500,945.209259,0.007600,16279.000000,0.000000
50%,0.057500,1895.000000,0.010300,16352.250000,0.000000
75%,0.057500,4000.000000,0.016000,16538.300781,1.000000
max,0.060000,26423.287109,0.019500,17051.900391,1.000000


Cek multikolinearitas dari variabel


In [15]:
df_saham['Close_lag'] = df_saham.groupby(level=0)['Close'].shift(1)
df_saham.dropna(inplace=True)
X = df_saham.drop(columns=['Close'])
X = add_constant(X)

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data.drop(index=0)

,Variable,VIF
1,BI-Rate,1.194842
2,Rate Inflasi,5.311394
3,USD_IDR,3.170057
4,Trump Tariffs,3.316234
5,Close_lag,1.001137


Untuk memastikan tidak terjadi multikolinearitas antar variabel independen, dilakukan perhitungan Variance Inflation Factor (VIF). Hasil menunjukkan bahwa seluruh variabel memiliki nilai VIF di bawah ambang batas umum 10, yang menandakan tidak adanya masalah multikolinearitas serius. Nilai VIF tertinggi terdapat pada variabel `Rate Inflasi` ($5,31$), yang mengindikasikan adanya korelasi sedang dengan variabel lain, namun masih dalam batas toleransi. Sementara itu, variabel lainnya seperti `BI-Rate` ($1,19$), `USD_IDR` ($3,17$), `Trump Tariffs` ($3,32$), dan `Close_lag` ($1,00$) menunjukkan tidak adanya indikasi multikolinearitas. Dengan demikian, seluruh variabel dapat digunakan dalam pemodelan regresi panel dinamis tanpa perlu dilakukan transformasi atau penghapusan.

In [19]:
df_saham['const'] = 1

mod = PanelOLS(
    dependent=df_saham['Close'],
    exog=df_saham[['const', 'Close_lag', 'BI-Rate', 'Rate Inflasi', 'USD_IDR', 'Trump Tariffs']],
    entity_effects=True,
)

res = mod.fit()
print(res.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  Close   R-squared:                        0.8739
Estimator:                   PanelOLS   R-squared (Between):              0.9942
No. Observations:                4005   R-squared (Within):               0.8739
Date:                Thu, Jun 05 2025   R-squared (Overall):              0.9934
Time:                        15:53:21   Log-likelihood                -2.541e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      5481.5
Entities:                          45   P-value                           0.0000
Avg Obs:                       89.000   Distribution:                  F(5,3955)
Min Obs:                       89.000                                           
Max Obs:                       89.000   F-statistic (robust):             5481.5
                            

Hasil estimasi menunjukkan bahwa model panel dinamis dengan memasukkan lag harga saham (`Close_lag`) sebagai variabel penjelas memberikan nilai R-squared sebesar $0,8739$ (within), yang menunjukkan bahwa model mampu menjelaskan sekitar $87\%$ variasi harga saham dalam setiap entitas (perusahaan). Variabel `Close_lag` berpengaruh sangat signifikan ($p < 0,01$) dengan koefisien $0,9227$, mengindikasikan adanya persistensi tinggi dalam harga saham. Variabel `BI-Rate` juga signifikan secara statistik ($p = 0,0206$), menunjukkan bahwa kenaikan suku bunga acuan Bank Indonesia berasosiasi dengan kenaikan harga saham, meskipun hal ini mungkin menunjukkan reaksi positif pasar terhadap stabilisasi moneter.

Variabel `USD_IDR` memiliki pengaruh negatif dan signifikan terhadap harga saham ($p < 0,01$), artinya depresiasi Rupiah terhadap Dolar AS berdampak buruk terhadap nilai saham perusahaan LQ45. Dummy `Trump Tariffs` juga signifikan positif ($p < 0,01$), dengan koefisien $33,49$, mengindikasikan bahwa penerapan tarif tersebut diasosiasikan dengan kenaikan harga saham, kemungkinan mencerminkan optimisme pasar domestik terhadap sektor-sektor yang lebih fokus pada pasar dalam negeri atau reaksi terhadap kebijakan fiskal lanjutan.

Sementara itu, variabel `Rate Inflasi` tidak signifikan pada tingkat konvensional ($p = 0,48$), yang menunjukkan bahwa dalam konteks ini, inflasi tidak memberikan pengaruh langsung yang kuat terhadap harga saham LQ45. Uji poolability menghasilkan p-value $< 0,01$, mendukung pemilihan model fixed effects dibanding pooled OLS.


In [20]:
residuals = res.resids
exog = df_saham[['const', 'Close_lag', 'BI-Rate', 'Rate Inflasi', 'USD_IDR', 'Trump Tariffs']]

bp_test = het_breuschpagan(residuals, exog)
print('Breusch-Pagan test statistic:', bp_test[0])
print('p-value:', bp_test[1])

Breusch-Pagan test statistic: 219.05108944307582
p-value: 2.372637489286354e-45


Hasil uji Breusch–Pagan menunjukkan nilai statistik sebesar $219,05$ dengan p-value $2,37×10⁻⁴⁵$, yang secara signifikan lebih kecil dari $0,05$. Karena p-value sangat rendah, hipotesis nol homoskedastisitas ditolak, sehingga terdapat bukti kuat adanya heteroskedastisitas pada error term. Untuk mengatasi masalah ini, disarankan menggunakan estimasi dengan standard error robust agar inference tetap valid.

In [21]:
dw_stat = durbin_watson(res.resids)
print('Durbin-Watson statistic:', dw_stat)

Durbin-Watson statistic: 2.0997435333156447


Nilai Durbin–Watson sebesar $2,0997$ menunjukkan bahwa residual model tidak terindikasi adanya autokorelasi serial pada lag pertama. Karena nilai ini sangat dekat dengan $2$ (nilai menunjukkan tidak ada autokorelasi), maka dapat disimpulkan tidak ada masalah signifikan terkait autokorelasi dalam residual.

# Kesimpulan

Analisis panel dinamis menunjukkan bahwa harga saham perusahaan dalam Indeks LQ45 memiliki tingkat persistensi yang tinggi, dengan koefisien lag harga penutupan sebesar $0,9227$ ($p < 0,01$). Hasil ini mengonfirmasi bahwa harga saham pada suatu hari sangat dipengaruhi oleh harga pada hari sebelumnya.

Variabel makroekonomi juga terbukti berpengaruh. Kenaikan suku bunga acuan Bank Indonesia (BI-Rate) berasosiasi positif dengan harga saham (koefisien $5681,2, p = 0,0206$), sedangkan depresiasi Rupiah terhadap Dolar AS (USD_IDR) memberikan dampak negatif yang signifikan (koefisien $–0,0659$, $p < 0,01$). Sementara itu, variabel inflasi tidak signifikan (p = 0.48) dalam memengaruhi harga saham LQ45 selama periode penelitian.

Variabel dummy “Trump Tariffs” menunjukkan pengaruh positif yang signifikan (koefisien $33,49$, $p < 0,01$), menandakan bahwa pengumuman dan penerapan tarif impor tahun 2025 cenderung meningkatkan harga saham di Indeks LQ45. Hal ini bisa mengindikasikan optimisme pelaku pasar terhadap perusahaan yang lebih terlindungi dari persaingan impor atau persepsi bahwa kebijakan tersebut tidak membebani sektor domestik secara langsung.

Hasil uji diagnostik mengungkapkan adanya heteroskedastisitas (Breusch–Pagan $p < 0,01$), sehingga perlu penggunaan standard error robust dalam estimasi agar validitas inferensi terjaga. Di sisi lain, uji Durbin–Watson ($2,0997$) menunjukkan tidak ada autokorelasi pada residual model.  

Secara keseluruhan, penelitian ini menegaskan bahwa kebijakan tarif impor Trump tahun 2025 memberikan dampak positif terhadap harga saham LQ45, meskipun dipengaruhi pula oleh kondisi makroekonomi domestik. Bagi investor dan pembuat kebijakan, penting untuk mempertimbangkan kondisi suku bunga dan nilai tukar ketika menilai efek kebijakan perdagangan internasional. Penelitian selanjutnya dapat mengeksplorasi efek jangka panjang dan sektor spesifik dalam LQ45 untuk pemahaman yang lebih mendalam.


# Sumber

[[1](https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration)] "Tariffs in the second Trump administration", Wikipedia, [Online]. Tersedia: https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration. [Diakses: Jun. 5, 2025]. \
[[2](https://id.wikipedia.org/wiki/Indeks_LQ45)] "Indeks LQ45", Wikipedia, [Online]. Tersedia: https://id.wikipedia.org/wiki/Indeks_LQ45. [Diakses: Jun. 5, 2025]. \
[[3](https://doi.org/10.24912/jmk.v4i1.17118)] A. B. Wijaya dan N. M. Nuryasman, "Pengaruh Variabel Makroekonomi dan Rasio Keuangan terhadap Harga Saham," Jurnal Manajerial dan Kewirausahaan, vol. 4, no. 1, pp. 12–21, Jan. 2022, doi: 10.24912/jmk.v4i1.17118. \
[4] E. W. Frees, Longitudinal and Panel Data: Analysis and Applications in the Social Sciences. Cambridge: Cambridge University Press, 2004.